In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [2]:
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [3]:
df = pd.read_csv('rbc_posneg.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)
df

,Dividends,High Change,Low Change,Vix ewm,High/Close,Low/Close,Vix Close,CAD Close,Spread,Spread Change,...,Vol change,Return ewm,Vix return,TD return,Tsx return,Cad return,Return,Chaikin,Chaikin Change,Pos
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-06,0.0,-2.263960e-02,-0.010593,-0.017666,0.006325,-0.004156,19.160000,1.03190,0.372903,-0.546876,...,0.301745,-0.010025,-0.009819,-0.009845,0.004744,-0.007311,-0.007354,3.729000e+05,-6.427948,1
2010-01-07,0.0,-3.052366e-03,-0.003447,-0.009368,0.002347,-0.008485,19.059999,1.03400,0.385765,0.034490,...,-0.365502,-0.002739,-0.005219,-0.008700,-0.004772,0.002035,0.000904,-4.557000e+05,-2.222043,0
2010-01-08,0.0,9.003467e-04,0.002549,-0.035652,0.005064,-0.004160,18.129999,1.02980,0.327897,-0.150006,...,-0.123795,-0.002117,-0.048793,-0.001253,0.005577,-0.004062,-0.001806,-3.658500e+05,-0.197169,0
2010-01-11,0.0,-3.993118e-08,0.000363,-0.033211,0.006885,-0.001993,17.549999,1.03380,0.315038,-0.039217,...,0.088201,-0.001911,-0.031991,0.007061,-0.000561,0.003884,-0.001809,-9.677500e+04,-0.735479,1
2010-01-12,0.0,-3.419030e-03,-0.006717,0.015520,0.000000,-0.012098,18.250000,1.03920,0.430770,0.367357,...,0.114357,0.001658,0.039886,-0.001247,-0.010622,0.005223,0.003443,7.316250e+04,-1.756006,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-13,0.0,-1.010593e-03,0.003524,-0.032076,0.001566,-0.003316,22.209999,1.27077,0.530006,-0.480384,...,-0.348914,0.001913,-0.048007,-0.002540,-0.002841,-0.004988,0.000922,-9.105427e+05,-0.040003,0
2021-01-14,0.0,7.357141e-04,-0.003512,0.020525,0.005824,-0.003328,23.250000,1.26981,0.989998,0.867898,...,1.585887,-0.001696,0.046826,0.013269,0.001305,-0.000755,-0.003500,8.358786e+05,-1.918000,1
2021-01-15,0.0,-3.216602e-03,-0.002968,0.038096,0.000553,-0.008303,24.340000,1.26477,0.959999,-0.030302,...,0.134256,0.000790,0.046882,-0.002381,-0.002734,-0.003969,0.002034,7.005893e+05,-0.161853,0


In [4]:
df.columns

Index(['Dividends', 'High Change', 'Low Change', 'Vix ewm', 'High/Close',
       'Low/Close', 'Vix Close', 'CAD Close', 'Spread', 'Spread Change',
       'Volume Diff', 'Vol change', 'Return ewm', 'Vix return', 'TD return',
       'Tsx return', 'Cad return', 'Return', 'Chaikin', 'Chaikin Change',
       'Pos'],
      dtype='object')

In [5]:
df_input = df[['Vix ewm', 'High/Close', 'Low/Close',
       'Vix Close', 'CAD Close', 'Spread', 'Volume Diff',
       'Return ewm', 'Vix return', 'TD return', 'Tsx return',
       'Return', 'Chaikin', "Dividends"]]
df_input

,Vix ewm,High/Close,Low/Close,Vix Close,CAD Close,Spread,Volume Diff,Return ewm,Vix return,TD return,Tsx return,Return,Chaikin,Dividends
Date,,,,,,,,,,,,,,
2010-01-06,-0.017666,0.006325,-0.004156,19.160000,1.03190,0.372903,814500.0,-0.010025,-0.009819,-0.009845,0.004744,-0.007354,3.729000e+05,0.0
2010-01-07,-0.009368,0.002347,-0.008485,19.059999,1.03400,0.385765,-1284300.0,-0.002739,-0.005219,-0.008700,-0.004772,0.000904,-4.557000e+05,0.0
2010-01-08,-0.035652,0.005064,-0.004160,18.129999,1.02980,0.327897,-276000.0,-0.002117,-0.048793,-0.001253,0.005577,-0.001806,-3.658500e+05,0.0
2010-01-11,-0.033211,0.006885,-0.001993,17.549999,1.03380,0.315038,172300.0,-0.001911,-0.031991,0.007061,-0.000561,-0.001809,-9.677500e+04,0.0
2010-01-12,0.015520,0.000000,-0.012098,18.250000,1.03920,0.430770,243100.0,0.001658,0.039886,-0.001247,-0.010622,0.003443,7.316250e+04,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-13,-0.032076,0.001566,-0.003316,22.209999,1.27077,0.530006,-872600.0,0.001913,-0.048007,-0.002540,-0.002841,0.000922,-9.105427e+05,0.0
2021-01-14,0.020525,0.005824,-0.003328,23.250000,1.26981,0.989998,2582300.0,-0.001696,0.046826,0.013269,0.001305,-0.003500,8.358786e+05,0.0
2021-01-15,0.038096,0.000553,-0.008303,24.340000,1.26477,0.959999,565300.0,0.000790,0.046882,-0.002381,-0.002734,0.002034,7.005893e+05,0.0


In [6]:
y = df[["Pos"]]
y.head()

,Pos
Date,
2010-01-06,1
2010-01-07,0
2010-01-08,0
2010-01-11,1
2010-01-12,1


In [7]:
x_train, x_test, y_train, y_test = train_test_split(df_input, y, test_size=0.3, random_state=42)

In [8]:
x_train_scaler = MinMaxScaler()
x_test_scaler = MinMaxScaler()


x_train_scaler.fit(x_train)
x_test_scaler.fit(x_test)


x_train = x_train_scaler.transform(x_train)
x_test = x_test_scaler.transform(x_test)

enc = OneHotEncoder()
enc.fit(y_train)

encoded_y_train = enc.transform(y_train).toarray()
encoded_y_test = enc.transform(y_test).toarray()
encoded_y_train[0]

array([0., 1.])

In [9]:
batch = 44
features = x_test.shape[1]

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout 

In [11]:
units = 64
drop = 0.2

nn = Sequential()

nn.add(Dense(units=units, input_dim=features, activation="relu"))

nn.add(Dense(2, activation="softmax"))

In [12]:
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                960       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 1,090
Trainable params: 1,090
Non-trainable params: 0
_________________________________________________________________


In [13]:
import tensorflow.keras.backend as K

def mean_pred(encoded_y_train, encoded_y_test):
    return K.mean(encoded_y_test)

In [14]:
nn.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [15]:
class ThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(ThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None): 
        val_loss = logs["val_accuracy"]
        if val_loss > self.threshold:
            self.model.stop_training = True

In [ ]:
earlystop = ThresholdCallback(threshold=0.58)

model = nn.fit(x_train, encoded_y_train, validation_data=(x_test, encoded_y_test), 
               epochs=500, callbacks=[earlystop], shuffle=True)

Train on 1901 samples, validate on 815 samples
Epoch 1/500
1901/1901 [==============================] - 1s 627us/sample - loss: 0.6889 - accuracy: 0.5565 - val_loss: 0.6920 - val_accuracy: 0.5313
Epoch 2/500
1901/1901 [==============================] - 0s 88us/sample - loss: 0.6861 - accuracy: 0.5592 - val_loss: 0.7002 - val_accuracy: 0.5252
Epoch 3/500
1901/1901 [==============================] - 0s 93us/sample - loss: 0.6864 - accuracy: 0.5571 - val_loss: 0.6984 - val_accuracy: 0.5227
Epoch 4/500
1901/1901 [==============================] - 0s 93us/sample - loss: 0.6855 - accuracy: 0.5581 - val_loss: 0.7011 - val_accuracy: 0.5252
Epoch 5/500
1901/1901 [==============================] - 0s 97us/sample - loss: 0.6849 - accuracy: 0.5539 - val_loss: 0.6944 - val_accuracy: 0.5239
Epoch 6/500
1901/1901 [==============================] - 0s 92us/sample - loss: 0.6856 - accuracy: 0.5639 - val_loss: 0.6994 - val_accuracy: 0.5215
Epoch 7/500
1901/1901 [==============================] - 0s 97us

In [ ]:
model_loss, model_accuracy = nn.evaluate(x_test, encoded_y_test, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
predicted = nn.predict(x_test)
predicted = enc.inverse_transform(predicted).flatten().tolist()
results = pd.DataFrame({
    "Actual": y_test.Pos.values,
    "Predicted": predicted
})
results.head(10)

In [ ]:
plt.plot(model.history["val_accuracy"])
plt.legend(["train"])
plt.show()

In [ ]:
max(model.history["val_accuracy"])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(results.Actual, results.Predicted))

In [ ]:
model_json = nn.to_json()
file_path = Path("rbc_model_divposneg.json")
with open(file_path, "w") as json_file:
    json_file.write(model_json)
    
file_path = "rbc_weights_divposneg.h5"
nn.save_weights(file_path)